In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage
from PIL import Image
from pytesseract import *
pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


In [ ]:
def plot_image(img, grayscale=True):
    plt.axis('off')
    if grayscale:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
img = cv2.imread(f"Carro1.jpg")
print(img.shape)
plot_image(img, False)

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print(gray.shape)
plot_image(gray)

In [ ]:
thresh = cv2.threshold(gray, 170, 255, cv2.THRESH_BINARY_INV)[1]
plot_image(thresh)

In [ ]:
contours = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
canvas = np.zeros_like(img)
cv2.drawContours(canvas, contours, -1, (0, 255 ,0), 2)
plt.axis("off")
plt.imshow(canvas);

In [ ]:
#Medidas de las placas colombianas para vehiculos: 300mmx160mm,
#aspect ratio = 1.875
license_ratio = 1.875
min_an = 282
max_an = 400
min_al = 200
max_al = 289
candidates = []
for cnt in contours:
    x,y,w,h = cv2.boundingRect(cnt)
    aspect_ratio = float(w)/h
    if (np.isclose(aspect_ratio, license_ratio, atol=3) and
       (max_an > w > min_an) and
       (max_al > w > min_al)):
        candidates.append(cnt)
canvas = np.zeros_like(img)
cv2.drawContours(canvas, candidates, -1, (0,255,0),2)
plt.axis("off")
plt.imshow(canvas)

In [ ]:
ys=[]
for cnt in candidates:
    x,y,w,h = cv2.boundingRect(cnt)
    ys.append(y)
license = candidates[np.argmax(ys)]

In [ ]:
x,y,w,h = cv2.boundingRect(license)
cropped = img[y:y+h,x+160:x+w]
plot_image(cropped, False)

In [ ]:
gray_cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
thresh_cropped = cv2.adaptiveThreshold(gray_cropped, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 7, 13)
plot_image(thresh_cropped)

In [ ]:
borderless = skimage.segmentation.clear_border(thresh_cropped)
plot_image(borderless)

In [ ]:
final = cv2.bitwise_not(borderless)
plot_image(final)

In [ ]:
alphanumeric = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
options = "-c tessedit_char_whitelist={}".format(alphanumeric)
options += " --psm 7"

In [ ]:
cv2.imwrite("Placa_guardada.png", final)

In [ ]:
SP = Image.open("Placa_guardada.png")

In [ ]:
resultado = pytesseract.image_to_string(SP, config=options)

In [ ]:
print(resultado)